给定一个问题和解决方案，先扩写它们，再生成专利背景介绍、权要和实施方式。

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
template_rewriteq = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
这个发明点属于流程挖掘产品，请根据发明人提供的简要描述和以下规则，重新描述专利发明点要解决的问题。
1，待解决的问题应用于什么样的业务场景？请给出一个简单且有代表性的例子，方便理解。
2，这类问题的一般解决思路是什么？这样的解决思路存在什么样的功能或者性能上的弱点或者缺陷？请给出技术上的理由。
3，为什么要解决这个问题？它有多大的现实价值？请结合业务场景和技术特征进行分析。
4，简要描述中不清楚或者忽略的地方是什么？如何描述，才能更加完整、准确地表示这个问题？

待解决问题的简单描述: {raw_question}

请根据以上规则要求，以更完整、更详细的方式重新描述要解决的问题：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [4]:
template_commentq = '''你是一个专业的流程挖掘产品经理，请评估更新版本的问题描述是否存在不完善的地方。
如果存在不完善的地方，请重新描述一下。如果没有，可以直接复用原来的问题描述。
1，更新版本是否表达了与原始版本相同的问题？如果有不一致的地方，请修正一下；
2，更新版本是否增加了更多的技术细节、应用场景，而不只是引用或者复述原始版本？如果未增加更多的细节，请补充；
3，更新版本是否解决了原始版本中未清晰表达的关键内容？如果没有很好的解决，请重新更正；
4，更新版本都包含了哪些核心概念和术语，请解释这些概念和术语，以及它们之间可能的逻辑关系。

原始版本的问题描述: {raw_question}
更新版本的问题描述: {question}

请在更新版本的基础上，修改存在的不足：'''

prompt_commentq = PromptTemplate(input_variables=['raw_question', 'question'], template=template_commentq)
commentq_chain = LLMChain(llm=llm, prompt=prompt_commentq, output_key='commentq')

In [5]:
template_candidates = '''你们是三个著名的程序员，请根据问题描述、有效性评估及用户提供的解决方案，各自提供一个初步的且具体创新性的解决方案。
解决方案要包含基本的解决思路，使用的算法和数据描述，以及具体的实现步骤。

待解决的问题: {question}
对该问题描述的有效性评估：{commentq}
用户提供的解决方案：{raw_solution}

请一步步思考，确保解决方案能够解决上述问题，并转换为具体的代码实现。
你们三个的解决方案分别是：'''

prompt_candidates = PromptTemplate(input_variables=['question', 'commentq', 'raw_solution'], template=template_candidates)
candidates_chain = LLMChain(llm=llm, prompt=prompt_candidates, output_key='candidates')

In [6]:
template_rewritea = '''你是一个专业的流程挖掘领域的专利代理，你的任务是帮助专利发明人完善发明点设计。
请根据待解决问题的描述，自动生成的解决方案和用户提供的解决方案，重新组合生成一个可以作为专利发明点的解决方案。
这个解决方案需要具体、详细，能够转换为代码实现，还要包含以下内容：
1，它的基本思路是什么？用到了哪些成熟算法和数据？请具体分析这个方法如何解决问题的。
2，它与一般的解决方法有什么不同？不管是算法上的不同，应用场景的不同，还是使用的数据不同都可以。
3，这些不同点对应的优势和价值是什么？如果创新点的价值不高，可以通过什么样的方法来提高？
4，它的具体解决步骤是什么？可以综合多个解决方案中的实需现，但要保证每个步骤都是必须的，而且能够解决全部问题。

待解决问题的描述: {question}
自动生成的解决方案: {candidates}
用户提供的解决方案：{raw_solution}

请根据上述方案，重新组合一个具备创新性的解决方案：'''

prompt_rewritea = PromptTemplate(input_variables=['question', 'candidates', 'raw_solution'], template=template_rewritea)
rewritea_chain = LLMChain(llm=llm, prompt=prompt_rewritea, output_key='solution')

In [7]:
template_background = '''你是一个专业的流程挖掘领域的专利代理，帮助专利发明人设计发明点设计，并起草专利草稿。请根据以下待解决问题及解决方案的描述，起草专利的背景介绍部分。
专利的背景介绍要包含所解决问题所在的技术领域介绍，问题的简单介绍，以及这个问题所产生的不良影响。
待解决问题的描述: {question}
解决方案的描述: {solution}
请生成这个专利发明点的背景介绍：'''

prompt_background = PromptTemplate(input_variables=['question', 'solution'], template=template_background)
background_chain = LLMChain(llm=llm, prompt=prompt_background, output_key='background')

In [8]:
template_claims = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的权利要求部分。
在设计专利的权利要求时要一步步来，先判断解决方案的创新性和价值所在，然后再根据创新点来设计对应的权利要求。针对要解决的问题，先将解决方案重新描述为多个步骤，再判断每一个步骤
是否有必要存在，若是没有必要，则可以删除；对于必要存在的步骤，再考虑它是否具备创新性。若是不具备，则更新对应的步骤。只要其中一个步骤具备创新性即可。
可使用以下问题来引导生成过程：

要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的权利要求：'''

prompt_claims = PromptTemplate(input_variables=['question', 'solution'], template=template_claims)
claims_chain = LLMChain(llm=llm, prompt=prompt_claims, output_key='claims')

In [9]:
template_explanation = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据解决的问题、解决方案及权利要求，
扩充权利要求部分的内容。扩充的方法是，先设计一个具体的应用场景，将权利要求转换为具体的实现步骤，并结合应用场景进行描述每一步的输入输出，以及中间的处理过程。
若是不同权利要求对应的操作步骤是相同的，则只需要在第一次给出描述，后面的则省略。
可使用这些问题来引导生成过程：它的主要算法或者技术是什么？需要什么样的软硬件运行环境？是否有多种实现方式？每种实现方式最简单的形式是什么样的？

要解决的问题: {question}
提供的解决方案: {solution}
独立权利要求: {claims}

请将权利要求扩展为具体的实现步骤，并给出相应的示例说明：'''

prompt_explanation = PromptTemplate(input_variables=['question', 'solution', 'claims'], template=template_explanation)
explanation_chain = LLMChain(llm=llm, prompt=prompt_explanation, output_key='explanation')

In [10]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[rewriteq_chain, commentq_chain, candidates_chain, rewritea_chain, background_chain, claims_chain, explanation_chain],
    input_variables=["raw_question", "raw_solution"],
    # Here we return multiple variables
    output_variables=["question", "commentq", "candidates", "solution", "background", "claims", "explanation"],
    verbose=True)

In [ ]:
question = '''为了还原多个用户共同的UI操作流程，需要识别流程中哪些UI操作是相同的，哪些是不同的。比如，提交报销单，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，
其他的比如出差事由、行程安排等内容相差非常大，从视觉上看起来也很不一样。但是，它们同属于一个业务操作，应该判定相同UI操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。
仅用图片相似性无法解决这个问题，因为文字内容差异难以用图片像素来检测；仅用文字比较也不行，因为文本内容相差很大的UI操作可能属于同一个UI操作。
'''

solution = '''使用CV算法识别窗口/应用的区域分割（即用边界线表示布局样式），同一个窗口的区域分割应该是完全一样的。除此之外，再提取一两个关键文本标签，代表当前操作的语义信息。
只要窗口区域分割相同、关键标签相同，就认为两个UI界面属于同一个UI操作。识别窗口区域分割的方法是canny算法，通过边框来确定窗口的布局。这里假定同一个应用内只有有限个窗口，并且每个窗口的布局是很少变化的。
提取关键文本标签，是指使用OCR识别窗口中的所有文字，然后提取位置或者内容确定的关键标签，比如窗口标题栏中的文字。两者结合就可以较有效地区分不同窗口，进而判断UI操作的相似性。
'''

result = overall_chain({'raw_question': question, 'raw_solution': solution})



> Entering new SequentialChain chain...

> Finished chain.


In [ ]:
print(result['question'])

本发明点解决的问题是如何识别多个用户共同的UI操作流程中哪些UI操作是相同的，哪些是不同的。这个问题在业务场景中非常普遍，例如，在一个报销系统中，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，其他的比如出差事由、行程安排等内容相差非常大，从视觉上看起来也很不一样。但是，它们同属于一个业务操作，应该判定为相同UI操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。

现有解决思路一般是基于图片相似性或文本内容来进行识别。然而，仅用图片相似性无法解决这个问题，因为文字内容差异难以用图片像素来检测；仅用文字比较也不行，因为文本内容相差很大的UI操作可能属于同一个UI操作。另外，现有的文本匹配算法可能会因为语义差异而导致匹配失败。

本发明点的解决思路是结合图片和文本的特征进行综合分析，采用流程挖掘的方法，通过对多个用户共同的UI操作流程进行分析和比较，找出相同的UI操作。具体来说，可以将UI操作流程转换为状态转移图，然后使用图匹配算法来识别相同的UI操作，同时结合文本内容进行进一步的分析。

本发明点的现实价值非常大。对于企业来说，能够准确识别相同的UI操作，可以提高系统的自动化程度，降低人工操作的错误率和工作量，提高工作效率。对于用户来说，能够快速找到相同的UI操作，可以减少学习成本和操作难度，提高用户体验。

简要描述中可能存在的不清楚或忽略的地方是，没有具体说明如何将UI操作流程转换为状态转移图，以及如何结合文本内容进行分析。需要进一步描述这些细节，以使得发明点的实施更加完整和准确。


In [ ]:
print(result['commentq'])

更新版本描述存在一些不足，需要进一步完善：

1. 在描述问题时，没有明确说明为什么现有的文本匹配算法会因为语义差异而导致匹配失败。需要进一步解释这个问题的本质原因，以使得读者更好地理解本发明点的解决思路。

2. 在描述解决思路时，没有具体说明如何将UI操作流程转换为状态转移图。需要进一步描述这个过程的具体步骤和算法原理，以使得读者能够清晰地理解这个过程。

3. 在描述解决思路时，也没有具体说明如何结合文本内容进行分析。需要进一步描述文本内容的抽取和分析方法，以及如何将文本内容与状态转移图进行结合，以实现更准确的UI操作识别。

针对以上不足，可以进行以下修改：

更新版本的问题描述：本发明点解决的问题是如何识别多个用户共同的UI操作流程中哪些UI操作是相同的，哪些是不同的。这个问题在业务场景中非常普遍，例如，在一个报销系统中，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，其他的比如出差事由、行程安排等内容相差非常大，从视觉上看起来也很不一样。但是，它们同属于一个业务操作，应该判定为相同UI操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。

现有解决思路一般是基于图片相似性或文本内容来进行识别。然而，仅用图片相似性无法解决这个问题，因为文字内容差异难以用图片像素来检测；仅用文字比较也不行，因为文本内容相差很大的UI操作可能属于同一个UI操作。另外，现有的文本匹配算法可能会因为语义差异而导致匹配失败。

本发明点的解决思路是结合图片和文本的特征进行综合分析，采用流程挖掘的方法，通过对多个用户共同的UI操作流程进行分析和比较，找出相同的UI操作。具体来说，可以将UI操作流程转换为状态转移图，然后使用图匹配算法来识别相同的UI操作，同时结合文本内容进行进一步的分析。具体步骤如下：

1. UI操作流程的转换：将UI操作流程转换为有向图，其中每个节点表示一个UI操作，每个边表示两个UI操作之间的关系。可以使用深度优先搜索（DFS）或广度优先搜索（BFS）等方法进行图的遍历和生成。

2. 图匹配算法：采用图匹配算法来识别相同的UI操作。可以使用子图同构算法、图编辑距离算法等方法来进行匹配。

3. 文本内容的抽取和分析：从UI操作中抽取文本内容，并使用自然语言处理技术进行分析。可以使用词向量模型、主题模型等方法来进行文本分析。

4. 结合图和文

In [ ]:
print(result['candidates'])

程序员A的解决方案：

基本解决思路：通过对多个用户共同的UI操作流程进行分析和比较，找出相同的UI操作。具体来说，可以将UI操作流程转换为状态转移图，然后使用图匹配算法来识别相同的UI操作，同时结合文本内容进行进一步的分析。

算法和数据描述：

1. UI操作流程的转换

首先，遍历UI操作流程，将每个UI操作节点作为有向图的一个节点，每个有向边表示两个UI操作之间的关系。这里我们采用深度优先搜索（DFS）算法来进行图的遍历和生成。

代码实现：

// 生成UI操作流程的有向图
class Node {
public:
    int id; // 节点id
    string info; // UI操作信息
    vector<Node*> next; // 后继节点
    Node(int _id, string _info) {
        id = _id;
        info = _info;
    }
};
class Solution {
public:
    void dfs(Node* node, unordered_map<int, bool>& visited, vector<Node*>& graph) {
        visited[node->id] = true;
        graph.push_back(node);
        for (auto n : node->next) {
            if (!visited[n->id]) {
                dfs(n, visited, graph);
            }
        }
    }
    vector<vector<Node*>> getUIOperationGraph(vector<Node*>& uiOperations) {
        vector<vector<Node*>> res;
        unordered_map<int, bool> visited;
        for (auto ui : uiOperations) {
            if (!visited[ui->id]) {
                vector<Node*

In [ ]:
print(result['solution'])

综合以上方案，我们提出一个具备创新性的解决方案，具体步骤如下：

1. 将UI操作流程转换为状态转移图，使用深度优先搜索算法进行图的遍历和生成。

2. 使用子图同构算法来进行UI操作的匹配，同时结合文本内容进行进一步的分析。使用词向量模型来计算文本相似度，筛选出相似度高于阈值的UI操作。

3. 使用CV算法识别窗口/应用的区域分割，同一个窗口的区域分割应该是完全一样的。提取关键文本标签，比如窗口标题栏中的文字。

4. 将UI操作转换为特征向量，使用聚类算法来进行分类，将相同的UI操作划分到同一类别中。这里可以使用k-means算法来进行聚类。

5. 将同一类别中的UI操作视为相同的UI操作，通过结合窗口布局和关键文本标签来判断UI操作的相似性。

这个方案的创新点在于将多种算法进行结合，既考虑了UI操作的图像特征，又考虑了文本内容的相似度，同时将窗口布局、关键文本标签和特征向量进行结合，提高了UI操作识别的准确度和效率。此外，该方法还可以应用于其他领域的UI操作识别和分类，具有广泛的应用价值。


In [ ]:
print(result['background'])

技术领域介绍：

本发明点涉及流程挖掘领域，尤其是在UI操作识别和分类的应用方面具有广泛的应用价值。随着信息技术的发展，越来越多的业务操作都在通过图形化用户界面（UI）进行，这使得UI操作的自动化识别和分类变得尤为重要。尽管已有多种UI操作识别算法，但是它们往往只考虑UI操作的部分特征，难以准确地识别相同的UI操作。因此，需要一种能够充分利用UI操作的多种特征进行综合分析的方法，以提高UI操作的识别准确度和效率。

问题简介：

本发明点解决的问题是如何识别多个用户共同的UI操作流程中哪些UI操作是相同的，哪些是不同的。这个问题在业务场景中非常普遍，例如，在一个报销系统中，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，其他的比如出差事由、行程安排等内容相差非常大，从视觉上看起来也很不一样。但是，它们同属于一个业务操作，应该判定为相同UI操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。

不良影响：

现有解决思路一般是基于图片相似性或文本内容来进行识别。然而，仅用图片相似性无法解决这个问题，因为文字内容差异难以用图片像素来检测；仅用文字比较也不行，因为文本内容相差很大的UI操作可能属于同一个UI操作。另外，现有的文本匹配算法可能会因为语义差异而导致匹配失败。这些问题将导致UI操作的识别准确度和效率降低，从而影响到系统的自动化程度、操作的错误率和工作量、用户体验等方面的不良影响。

解决方案：

本发明点提出了一种具有创新性的UI操作识别和分类方法。该方法综合考虑了UI操作的多种特征，包括图像特征和文本特征，同时结合窗口布局和关键文本标签等信息，通过流程挖掘的方法将UI操作流程转换为状态转移图，并使用子图同构算法来进行UI操作的匹配。具体步骤包括将UI操作流程转换为状态转移图，使用深度优先搜索算法进行图的遍历和生成；使用词向量模型来计算文本相似度，筛选出相似度高于阈值的UI操作；使用CV算法识别窗口/应用的区域分割，提取关键文本标签；将UI操作转换为特征向量，使用聚类算法来进行分类，将相同的UI操作划分到同一类别中；将同一类别中的UI操作视为相同的UI操作，通过结合窗口布局和关键文本标签来判断UI操作的相似性。该方法具有较高的准确度和效率，适用于多种UI操作识别和分类的应用场景。


In [ ]:
print(result['claims'])

1. 一种UI操作流程识别方法，包括以下步骤：
将UI操作流程转换为状态转移图；
使用子图同构算法进行UI操作匹配；
结合文本内容进行进一步的分析，使用词向量模型计算文本相似度，筛选出相似度高于阈值的UI操作；
使用CV算法识别窗口/应用的区域分割，提取关键文本标签；
将UI操作转换为特征向量，使用聚类算法进行分类，将相同的UI操作划分到同一类别中；
将同一类别中的UI操作视为相同的UI操作。

2. 根据权利要求1所述的UI操作流程识别方法，进一步地，在进行UI操作匹配时，使用深度优先搜索算法进行图的遍历和生成。

3. 根据权利要求1所述的UI操作流程识别方法，进一步地，在进行UI操作匹配时，采用阈值筛选法，筛选出相似度高于阈值的UI操作。

4. 根据权利要求1所述的UI操作流程识别方法，进一步地，在进行窗口/应用的区域分割时，使用CV算法识别窗口/应用的区域分割，提取关键文本标签。

5. 根据权利要求1所述的UI操作流程识别方法，进一步地，在进行UI操作分类时，使用k-means算法进行聚类。

6. 根据权利要求1所述的UI操作流程识别方法，进一步地，在进行UI操作相似性判断时，结合窗口/应用布局和关键文本标签来判断UI操作的相似性。


In [ ]:
print(result['explanation'])

权利要求1所述的UI操作流程识别方法的具体实现步骤如下：

1.1 将UI操作流程转换为状态转移图：

例如，在一个报销系统中，用户填写报销单的操作流程可以分为以下几个步骤：

打开报销系统 -> 填写报销单基本信息 -> 填写出差事由 -> 填写行程安排 -> 提交报销单

将这些操作流程转化为状态转移图的过程如下：

1.2 使用子图同构算法进行UI操作匹配：

对于两个UI操作流程，可以将其转化为状态转移图，然后使用子图同构算法来进行匹配。例如，假设两个用户在报销系统中填写报销单的操作流程如下：

UI操作流程1：打开报销系统 -> 填写报销单基本信息 -> 填写出差事由 -> 填写行程安排 -> 提交报销单

UI操作流程2：打开报销系统 -> 填写报销单基本信息 -> 填写出差事由 -> 填写备注 -> 提交报销单

对于这两个UI操作流程，可以将其转化为状态转移图，然后使用子图同构算法来进行匹配。通过比对两个状态转移图，可以发现它们的基本结构是相同的，只是在填写行程安排和填写备注这两个步骤上有所不同。

1.3 结合文本内容进行进一步的分析：

在进行UI操作匹配时，还需要结合文本内容进行进一步的分析。例如，在报销系统中，有些操作步骤可能只是文本内容有所不同，但是实际上它们是相同的操作。比如，填写报销单基本信息和填写出差事由这两个操作步骤，它们的UI界面可能不同，但是它们都属于填写报销单的基本信息。因此，在进行UI操作匹配时，需要使用词向量模型计算文本相似度，筛选出相似度高于阈值的UI操作。

1.4 使用CV算法识别窗口/应用的区域分割，提取关键文本标签：

在进行UI操作匹配时，还需要识别窗口/应用的区域分割，提取关键文本标签。可以使用CV算法来识别窗口/应用的区域分割，然后提取关键文本标签，比如窗口标题栏中的文字。

1.5 将UI操作转换为特征向量，使用聚类算法进行分类，将相同的UI操作划分到同一类别中：

在进行UI操作分类时，可以将UI操作转换为特征向量，使用聚类算法进行分类，将相同的UI操作划分到同一类别中。这里可以使用k-means算法来进行聚类。

1.6 将同一类别中的UI操作视为相同的UI操作：

最后，将同一类别中的UI操作视为相同的UI操作，通过结合窗口布局和关键文本标签来判断UI操作的相似性。例如，在报销系统中，将填写报销单基

In [ ]:
with open('patents/patent-rpm-test2f.txt', 'w') as f:
    f.write('>>> original question: \n' + result['raw_question'])  
    f.write('\n\n>>> original solution: \n' + result['raw_solution'])
    f.write('\n\n>>> revised question: \n' + result['question'])
    f.write('\n\n>>> commented question: \n' + result['commentq'])
    f.write('\n\n>>> solution candidates: \n' + result['candidates'])
    f.write('\n\n>>> final solution: \n' + result['solution'])
    f.write('\n\n>>> background: \n' + result['background'])
    f.write('\n\n>>> claims: ' + result['claims'])
    f.write('\n\n>>> explanation: ' + result['explanation'])